code to generate the sortability graph of r-uniform sock sequences

In [83]:
import math
import string
from datetime import datetime
alphabet = list(string.ascii_lowercase)
print(alphabet)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [35]:
def sequence_to_partition(p):
    """
    input: p (str or list) - sock ordering
    returns: partition (set) - corresponding set partition
    """
    sock_to_index = {} 
    for i in range(len(p)):
        sock = p[i]
        sock_to_index.setdefault(sock, set()).add(i)
    for sock in sock_to_index:
        sock_to_index[sock] = frozenset(sock_to_index[sock])
    partition = list(sock_to_index.values())
    return partition

def get_partition_size(partition):
    size = 0
    for subset in partition:
        size += len(subset)
    return size

def partition_to_standard_seq(partition):
    N = get_partition_size(partition)
    standard_seq = [0] * N
    for i in range(len(partition)):
        for idx in partition[i]:
            standard_seq[idx] = alphabet[i]
    standard_seq_str = ''
    for sock in standard_seq:
        standard_seq_str += str(sock)
    return standard_seq_str

def get_standardized(p):
    return partition_to_standard_seq(sequence_to_partition(p))

In [43]:
print(sequence_to_partition('ccbbdcd'))
print(get_partition_size(sequence_to_partition('ccbbdcd')))
print(partition_to_standard_seq(sequence_to_partition('ccbbdcd')))

[frozenset({0, 1, 5}), frozenset({2, 3}), frozenset({4, 6})]
7
aabbcac


In [45]:
from sage.combinat.permutation import Permutations_mset
mset = ['a', 'b', 'c', 'a', 'b', 'c']
perms = Permutations_mset(mset)
len(perms)
for perm in perms:
    print(perm)

['a', 'a', 'b', 'b', 'c', 'c']
['a', 'a', 'b', 'c', 'b', 'c']
['a', 'a', 'b', 'c', 'c', 'b']
['a', 'a', 'c', 'b', 'b', 'c']
['a', 'a', 'c', 'b', 'c', 'b']
['a', 'a', 'c', 'c', 'b', 'b']
['a', 'b', 'a', 'b', 'c', 'c']
['a', 'b', 'a', 'c', 'b', 'c']
['a', 'b', 'a', 'c', 'c', 'b']
['a', 'b', 'b', 'a', 'c', 'c']
['a', 'b', 'b', 'c', 'a', 'c']
['a', 'b', 'b', 'c', 'c', 'a']
['a', 'b', 'c', 'a', 'b', 'c']
['a', 'b', 'c', 'a', 'c', 'b']
['a', 'b', 'c', 'b', 'a', 'c']
['a', 'b', 'c', 'b', 'c', 'a']
['a', 'b', 'c', 'c', 'a', 'b']
['a', 'b', 'c', 'c', 'b', 'a']
['a', 'c', 'a', 'b', 'b', 'c']
['a', 'c', 'a', 'b', 'c', 'b']
['a', 'c', 'a', 'c', 'b', 'b']
['a', 'c', 'b', 'a', 'b', 'c']
['a', 'c', 'b', 'a', 'c', 'b']
['a', 'c', 'b', 'b', 'a', 'c']
['a', 'c', 'b', 'b', 'c', 'a']
['a', 'c', 'b', 'c', 'a', 'b']
['a', 'c', 'b', 'c', 'b', 'a']
['a', 'c', 'c', 'a', 'b', 'b']
['a', 'c', 'c', 'b', 'a', 'b']
['a', 'c', 'c', 'b', 'b', 'a']
['b', 'a', 'a', 'b', 'c', 'c']
['b', 'a', 'a', 'c', 'b', 'c']
['b', 'a

# functions for checking sortability

In [31]:
def check_sorted(p):
    """
    input: sock_orders (set) - set of sock orders
    returns: answer (bool) - whether or not sock_orders contains the unique sorted sock order
    """
    colors = set()
    i = 0
    while i in range(len(p)):
        current_color = p[i]
#         print(f'current color = {current_color}')
        if current_color not in colors:
            colors.add(current_color)
            while i in range(len(p)) and p[i] == current_color:
                i += 1
#                 print(i)
        else:
            return False
    return True

def contains_sorted(sock_orders):
    """
    input: sock_orders (set) - set of sock orders
    returns: answer (bool) - whether or not sock_orders contains the unique sorted sock order
    """
    answer = False
    for sock_order in sock_orders:
        if check_sorted(sock_order):
            answer = True
    return answer

In [42]:
print(check_sorted('aaabcd'))
print(check_sorted('aaabcad'))

True
False


# functions for BFS-ing sock orders

In [68]:
def dyck_word_to_sorting(p, dyck_word):
    """
    inputs: p (str) - sock ordering, dyck_word
    returns: resulting sock ordering from applying the stack sort corresponding to dyck_word 
    """
    stack = []
    p_seq = list(p)
    result = '' # resulting sock ordering
    
    for step in dyck_word:
        if step == 1: # push
            stack.append(p_seq.pop(0))
        else: # pop
            last_sock = stack.pop()
            result += last_sock
    return result
    
# compute all possible sequences in foot(p)

def foot(p):
    """
    input: p - str, sock ordering
    returns: set of all (standardized) sock orderings achievable from stack-sorting p once
    """
    # always standardized
    N = len(p)
    sock_orders = set()
    dyck_words_list = DyckWords(N).list()
    for dyck_word in dyck_words_list:
        sock_orders.add(get_standardized(dyck_word_to_sorting(p, dyck_word)))
    return sock_orders # not necessarily injective from dyck_words to orders

def foot_k_full(p, k):
    # make every sock order standardized
    p_stand = get_standardized(p)
    parents = {p_stand:None}
    levels = [[p_stand]]
    
    while len(levels[-1]) > 0 and len(levels) <= k:
        levels.append([])
        for u in levels[-2]:
            for v in foot(u):
                v_stand = get_standardized(v)
                if v_stand not in parents:
                    levels[-1].append(v_stand)
                    parents[v_stand] = u
                
    return parents, levels

def foot_k_search(p,k):
    p_stand = get_standardized(p)
    parents = {p_stand:None}
    levels = [[p_stand]]
    if check_sorted(p_stand):
        return p_stand
    
    while len(levels[-1]) > 0 and len(levels) <= k:
        levels.append([])
        for u in levels[-2]:
            for v in foot(u):
                v_stand = get_standardized(v)
                if v_stand not in parents:
                    levels[-1].append(v_stand)
                    parents[v_stand] = u
                    if check_sorted(v_stand):
                        return v_stand
    return None # if no sorted found within distance k
    
#     def foot_k_search_helper(p,k): # DFS
#         # returns sorted sock ordering, otherwise none
#         p_stand = get_standardized(p) # shouldn't need to standardize but test out bugs
#         if k == 0: # base case
#             if check_sorted(p_stand):
#                 return p_stand
#             return None
        
#         if check_sorted(p_stand):
#             return p_stand
#         for v in foot(u):
#             if v not in visited: # don't look at visited nodes - never helps to search in cycle
#                 v_sortability = foot_k_search_helper(v, k-1):
#                 if v_sortability:
#                     return v_sortability    

In [69]:
foot_k('aab', 2)

({'aab': None, 'aba': 'aab', 'abb': 'aab'}, [['aab'], ['aba', 'abb'], []])

In [72]:
print(foot_k_search('ababab', 1))

aaabbb


Current bound for $(n,r)$-ordering ($n$ colors, $r$-uniform) is that you need at least $\lfloor((r-1/r)\log_4(n)\rfloor$ stacks. For $r=2$, this is equal to $\lfloor(1/2\log_4(n)\rfloor$. Try different $n$, try to find $(r,n)$ ordering that needs more than and see where this breaks?

# functions for checking t-sortability

In [73]:
# r = 2, n = 2

def get_uniformity(p):
    """
    input: sock_orders (set) - set of sock orders
    returns: n, r (tup) assuming that p is a uniform ordering
    """
    n = len(set(p))
    r = int(len(p)/n)
    return n,r

# def k_sortable(p, k):
#     """
#     input: p (str) - sock ordering
#     returns: answer (bool) - whether or not you can sort in k stacks"""
#     n,r = get_uniformity(p)
#     sock_orders = foot_k(p, k)
# #     print(f'sock orders = {sock_orders}')
#     return contains_sorted(sock_orders)

def sortable_in_log4_stacks(p):
    """
    input: p (str) - sock ordering
    returns: answer (bool) - whether or not you can sort in (r-1/r)log4(n) stacks"""
    n,r = get_uniformity(p)
    candidate_stack_num = math.floor((r-1)/r*math.log(n, 4))
    print(f'candidate stack number = {candidate_stack_num}')
    if foot_k_search(p, candidate_stack_num):
        return True
    return False

def find_min_stacks(p):
    for i in range(100):
        if foot_k_search(p, i):
            return i
    return 'not sortable in 100 stacks...'

In [74]:
find_min_stacks('abcabcabc')

2

# functions for generating sock orderings

In [75]:
def get_sock_orders(m_set): # given input multiset m_set, generate all possible sock orders
    return Arrangements(m_set, len(m_set)).list()   

# functions for generating multisets quickly
def uniform_sock_orders(r, n): # r socks of n colors
    m_set = []
    for i in range(n):
        for j in range(r):
            m_set.append(alphabet[i])
    return get_sock_orders(m_set)
    

def fast_sock_orders(socks): # input mults is list of 2 lists, first list = colors, second = multiplicities
    m_set = []
    mults = socks[0]
    
    if len(socks) == 1: # default is to make colors in alphabetical order
        for i in range(len(mults)):
            for j in range(mults[i]):
                m_set.append(alphabet[i])       
        return get_sock_orders(m_set)
     
    colors = socks[1]
    assert len(colors) == len(mults), 'given # of socks must be same as given # of multiplicities'
    for i in range(len(mults)):
        for j in range(mults[i]):
            m_set.append(colors[i])   
    return get_sock_orders(m_set)

In [62]:
# EXAMPLES

# sock_orders(['a', 'a', 'b', 'b', 'c', 'c'])
uniform_sock_orders(3,3)
# fast_sock_orders([[2, 2, 1], ['b', 'c', 'd']])
# fast_sock_orders([[2, 2, 1]])

[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'c'],
 ['a', 'a', 'a', 'b', 'b', 'c', 'b', 'c', 'c'],
 ['a', 'a', 'a', 'b', 'b', 'c', 'c', 'b', 'c'],
 ['a', 'a', 'a', 'b', 'b', 'c', 'c', 'c', 'b'],
 ['a', 'a', 'a', 'b', 'c', 'b', 'b', 'c', 'c'],
 ['a', 'a', 'a', 'b', 'c', 'b', 'c', 'b', 'c'],
 ['a', 'a', 'a', 'b', 'c', 'b', 'c', 'c', 'b'],
 ['a', 'a', 'a', 'b', 'c', 'c', 'b', 'b', 'c'],
 ['a', 'a', 'a', 'b', 'c', 'c', 'b', 'c', 'b'],
 ['a', 'a', 'a', 'b', 'c', 'c', 'c', 'b', 'b'],
 ['a', 'a', 'a', 'c', 'b', 'b', 'b', 'c', 'c'],
 ['a', 'a', 'a', 'c', 'b', 'b', 'c', 'b', 'c'],
 ['a', 'a', 'a', 'c', 'b', 'b', 'c', 'c', 'b'],
 ['a', 'a', 'a', 'c', 'b', 'c', 'b', 'b', 'c'],
 ['a', 'a', 'a', 'c', 'b', 'c', 'b', 'c', 'b'],
 ['a', 'a', 'a', 'c', 'b', 'c', 'c', 'b', 'b'],
 ['a', 'a', 'a', 'c', 'c', 'b', 'b', 'b', 'c'],
 ['a', 'a', 'a', 'c', 'c', 'b', 'b', 'c', 'b'],
 ['a', 'a', 'a', 'c', 'c', 'b', 'c', 'b', 'b'],
 ['a', 'a', 'a', 'c', 'c', 'c', 'b', 'b', 'b'],
 ['a', 'a', 'b', 'a', 'b', 'b', 'c', 'c'

In [64]:
sock_orders = {'aba', 'aab'}
contains_sorted(sock_orders)

True

In [77]:
# r = 2, n = 4 - only get 0 stacks, so can take any unsorted sock ordering
# sortable_in_log4_stacks('aabbccdd')
# 1 stack - requires at least 16 colors
# r = 2, n = 9 - still only get 0 stacks, so can take any unsorted sock ordering
sortable_in_log4_stacks('aabbccddeeffgghhiia')

candidate stack number = 0


False

In [81]:
def stacks_needed(r, n):
    # return max number of stacks needed to sort r, n uniform sock ordering
    # as well as the sock order that achieved this 
    sock_orders = uniform_sock_orders(r, n)
    max_stacks = 0
    bad_sock_order = None
    for sock_order in sock_orders:
        for i in range(math.ceil(math.log(n, 2)) + 1): # ceil(log_2(n)) stacks always suffices
            if foot_k_search(sock_order, i):
                if i > max_stacks:
                    max_stacks = i
                    bad_sock_order = sock_order
                break
    return max_stacks, bad_sock_order

In [82]:
# fixing r = 2 first
for n in range(1,10):
    max_stacks, bad_sock_order = stacks_needed(2,n)
    print(f'max stacks = {max_stacks}')
    print(f'bad sock order = {bad_sock_order}')
    print(f'expected floor(1/2 * log4(n)) = {math.floor(1/2 * math.log(n, 4))}')
    print('--------')


max stacks = 0
bad sock order = None
expected floor(1/2 * log4(n)) = 0
--------
max stacks = 1
bad sock order = ['a', 'b', 'a', 'b']
expected floor(1/2 * log4(n)) = 0
--------
max stacks = 1
bad sock order = ['a', 'a', 'b', 'c', 'b', 'c']
expected floor(1/2 * log4(n)) = 0
--------
max stacks = 2
bad sock order = ['a', 'b', 'c', 'a', 'd', 'b', 'c', 'd']
expected floor(1/2 * log4(n)) = 0
--------


KeyboardInterrupt: 

In [207]:
foot(['a','b','c'])

{'abc', 'acb', 'bac', 'bca', 'cba'}